# Penumbra Calibration

This script calibrates the beam edge sharpness (penumbra width) for both MLC and Jaw directions.

**What you need:**
- Measured 80%-20% penumbra widths at 10 cm depth for multiple field sizes

**What to do:**
1. Enter your measured penumbra values in the table (lines 6-11)
2. Adjust `PENUMBRA_FWHM_MLCS` and `PENUMBRA_FWHM_JAWS` until calculated values match your measurements
3. Record the final values for your machine configuration

**Output:**
- Optimal `penumbra_fwhm` values [MLC, Jaw] in mm

In [ ]:
import torch
import numpy as np
from pydose_rt import DoseEngine
from pydose_rt.data import MachineConfig, Phantom, Beam

In [ ]:
# ============================================
# USER INPUT: Penumbra measurements (mm)
# ============================================
FIELD_SIZES_MM = [50, 100, 150, 200, 300]  # Field sizes in mm

# Measured 80%-20% penumbra widths at 10 cm depth (mm)
PENUMBRA_MLC_MEASURED = [4.73, 5.23, 5.50, 5.95, 6.49]
PENUMBRA_JAW_MEASURED = [4.04, 4.55, 4.82, 5.11, 5.59]


In [ ]:
# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float32

# Initial guesses for penumbra FWHM
PENUMBRA_FWHM_MLC_GUESS = 3.0  # mm
PENUMBRA_FWHM_JAW_GUESS = 1.5  # mm

# Create water phantom for testing
phantom_resolution = (0.5, 2.0, 0.5)  # mm (crossline, depth, inline)
phantom_shape = (800, 100, 800)  # voxels
phantom = Phantom.from_uniform_water(
    shape=phantom_shape, 
    spacing=phantom_resolution
).to(device).to(dtype)

# Test penumbra settings
penumbra_results = []
for field_size_mm in FIELD_SIZES_MM:
    # Create machine config with test penumbra
    machine_config = MachineConfig(
        preset="../../src/pydose_rt/data/machine_presets/umea_10MV.json",
        penumbra_fwhm=[PENUMBRA_FWHM_MLC_GUESS, PENUMBRA_FWHM_JAW_GUESS],
        head_scatter_amplitude=None,  # Skip for now
        head_scatter_sigma=None,  # Skip for now
        profile_corrections=None,  # Skip for now
        output_factors=None  # Skip for now
    )
    
    # Create beam
    beam = Beam.create(
        gantry_angle_deg=0.0,
        number_of_leaf_pairs=60,
        collimator_angle_deg=0.0,
        field_size_mm=(field_size_mm, field_size_mm),
        iso_center=(200, 100, 200),  # mm
        device=device,
        dtype=dtype
    )
    
    # Create dose engine
    dose_engine = DoseEngine(
        machine_config,
        kernel_size=401,
        dose_grid_spacing=phantom.resolution,
        dose_grid_shape=phantom.density_image.shape,
        beam_template=beam,
        device=device,
        dtype=dtype,
        adjust_values=False
    )
    
    # Compute dose
    dose = dose_engine.compute_dose(beam, density_image=phantom.density_image)
    dose_np = dose[0, :, 50, :].cpu().detach().numpy()
    
    # Extract profiles
    mlc_profile = dose_np[400, :]
    jaw_profile = dose_np[:, 400]
    
    # Calculate penumbra (80%-20% width)
    mlc_20 = 0.2 * mlc_profile.max()
    mlc_80 = 0.8 * mlc_profile.max()
    jaw_20 = 0.2 * jaw_profile.max()
    jaw_80 = 0.8 * jaw_profile.max()
    
    mlc_penumbra = (np.sum((mlc_profile > mlc_20) & (mlc_profile < mlc_80)) / 2) * phantom_resolution[0]
    jaw_penumbra = (np.sum((jaw_profile > jaw_20) & (jaw_profile < jaw_80)) / 2) * phantom_resolution[2]
    
    print(f"Field {field_size_mm}×{field_size_mm} mm: MLC={mlc_penumbra:.2f} mm, Jaw={jaw_penumbra:.2f} mm")
    penumbra_results.append((mlc_penumbra, jaw_penumbra))

# Adjust PENUMBRA_FWHM_MLC_GUESS and PENUMBRA_FWHM_JAW_GUESS iteratively
# until calculated penumbras match your measurements
